We've discussed three different approaches to calculating the optimal weighting matrix in GMM problems, and developed a Monte Carlo Framework in a Jupyter notebook to explore the finite sample properties of one of these (the two-step estimator) in the linear IV case.   This discussion involves a comparison of these three different approaches.

1. Using the notation developed in lecture and in the notebook, what can we say about the finite sample performance of the two-step, iterated, and continuously updated GMM estimators when $k=\ell$?  
2. Using the framework of the Jupyter notebook, what can you say about the behavior of the three estimators?  Fix a given $(N,\ell,\mbox{truth})$ and compare the bias and efficiency of these estimators.
3. Choose a parameter to vary (of the parameters given in the previous question) and explore how the behavior of the different estimators changes with that parameter (e.g., at what rate do the estimators converge as $N\rightarrow\infty$).  
4. Sometimes we're willing to assume prior knowledge about the error structure of a given problem which we can exploit in estimating our weighting matrix.  For example, the Monte Carlo DGP in the notebook has homoskedastic $u$, but this is not something we assumed we knew in constructing the optimal weighting matrix.  If we exploited this prior knowledge, how could this change our estimator for the weighting matrix?
5. Write an alternative function for estimating the optimal weighting matrix in the homoskedastic case.  How does this affect the empirical distribution of the estimates in the Monte Carlo experiment?
6. What if we had heteroskedasticity of the form $\mbox{E}(u_j^2|Z_j) = (Z_j^\top Z_j)\sigma^2$ for all observations $j$ (note that here the $Z_j$ is taken to be a column vector)?   Is this consistent with IV assumptions?   What would be the form of the optimal weighting matrix?  How might you implement an estimator of the weighting matrix that exploited this knowledge?  Can you think of a simple case in which this might be a reasonable model of the error structure?

Start by shamelessly stealing Ethan's code

In [1]:
import numpy as np
from numpy.linalg import inv

## Play with us!
beta = 1     # "Coefficient of interest"
gamma = 1    # Governs effect of u on X
sigma_u = 1  # Note assumption of homoskedasticity
## Play with us!

# Let Z have order ell, and X order 1, with Var([X,Z]|u)=VXZ

ell = 4 # Play with me too!

# Arbitrary (but deterministic) choice for VXZ
A = np.sqrt(1/np.arange(1,(ell+1)**2+1)).reshape((ell+1,ell+1)) 


## Below here we're less playful.

# Var([X,Z]|u) is constructed so that pos. def.
VXZ = A.T@A 

Q = VXZ[1:,[0]]  # EZX'

truth = (beta,gamma,sigma_u,VXZ)

## But play with Omega if you want to introduce heteroskedascity
Omega = (sigma_u**2)*VXZ[1:,1:] # E(Zu)(u'Z')

# Asymptotic variance of optimally weighted GMM estimator:
print(inv(Q.T@inv(Omega)@Q))

[[0.73115726]]


In [2]:
from scipy.stats import distributions as iid

def dgp(N,beta,gamma,sigma_u,VXZ):
    """Generate a tuple of (y,X,Z).

    Satisfies model:
        y = X@beta + u
        E Z'u = 0
        Var(u) = sigma^2
        Cov(X,u) = gamma*sigma_u^2
        Var([X,Z}|u) = VXZ
        u,X,Z mean zero, Gaussian

    Each element of the tuple is an array of N observations.

    Inputs include
    - beta :: the coefficient of interest
    - gamma :: linear effect of disturbance on X
    - sigma_u :: Variance of disturbance
    - VXZ :: Cov([X,Z|u])
    """
    
    u = iid.norm.rvs(size=(N,1))*sigma_u

    # "Square root" of VXZ via eigendecomposition
    lbda,v = np.linalg.eig(VXZ)
    SXZ = v@np.diag(np.sqrt(lbda))

    # Generate normal random variates [X*,Z]
    XZ = iid.norm.rvs(size=(N,VXZ.shape[0]))@SXZ.T

    # But X is endogenous...
    X = XZ[:,[0]] + gamma*u
    Z = XZ[:,1:]

    # Calculate y
    y = X*beta + u

    return y,X,Z

In [3]:
def gj(b,y,X,Z):
    """Observations of g_j(b).

    This defines the deviations from the predictions of our model; i.e.,
    e_j = Z_ju_j, where EZ_ju_j=0.

    Can replace this function to testimate a different model.
    """
    return Z*(y - X*b)

In [4]:
def gN(b,data):
    """Averages of g_j(b).

    This is generic for data, to be passed to gj.
    """
    e = gj(b,*data)

    # Check to see more obs. than moments.
    assert e.shape[0] > e.shape[1]
    
    return e.mean(axis=0)

In [5]:
def Omegahat(b,data):
    e = gj(b,*data)

    # Recenter! We have Eu=0 under null.
    # Important to use this information.
    e = e - e.mean(axis=0) 
    
    return e.T@e/e.shape[0]

In [6]:
def J(b,W,data):

    m = gN(b,data) # Sample moments @ b
    N = data[0].shape[0]

    return N*m.T@W@m # Scale by sample size

In [7]:
from scipy.optimize import minimize_scalar

def two_step_gmm(data):

    # First step uses identity weighting matrix
    W1 = np.eye(gj(1,*data).shape[1])

    b1 = minimize_scalar(lambda b: J(b,W1,data)).x 

    # Construct 2nd step weighting matrix using
    # first step estimate of beta
    W2 = inv(Omegahat(b1,data))

    return minimize_scalar(lambda b: J(b,W2,data))

In [8]:
def cont_updated_gmm(data):
    return minimize_scalar(lambda b: J(b,inv(Omegahat(b,data)),data))

In [23]:
max_iterations = 10000
sensitivity = 0.01
def iterated_gmm(data):
    
    # First step uses identity weighting matrix
    Wn = np.eye(gj(1,*data).shape[1])

    # Iterate until Wn converges
    for n in range(max_iterations):   # restrict number of iterations
        bn = minimize_scalar(lambda b: J(b,Wn,data)).x 
        Wn_new = inv(Omegahat(bn,data))
        diff_mat = Wn - Wn_new # calculate difference matrix
        max_diff = np.amax(np.absolute(diff_mat)) # find biggest element, in abs value
        if max_diff <= sensitivity: # if the biggest difference is small enough, return matrix
            return minimize_scalar(lambda b: J(b,Wn_new,data))
        else:
            Wn = Wn_new # if not, reassign Wn to be the new mat and repeat
    
    return minimize_scalar(lambda b: J(b,Wn,data))

In [9]:
# Draw some data
N = 1000
data = dgp(N,beta,gamma,sigma_u,VXZ)